# Week 2: Spike-Stimulus Analysis

This notebook covers methods for analyzing the relationship between neural spiking
and behavioral variables. We'll focus on **place cells** - hippocampal neurons that
fire preferentially when an animal is at specific locations.

Using the same dataset from Week 1 ([Petersen & Buzsáki, 2020](https://pmc.ncbi.nlm.nih.gov/articles/PMC7442698/)),
we'll examine how neural firing relates to the animal's position during spatial navigation.

## Learning Objectives

By the end of this notebook, you will be able to:

1. Load position data and spike times from an NWB file
2. Visualize a neuron's spatial firing pattern
3. Compute and interpret a place field

## Setup

In [1]:
# Install dependencies (required for Google Colab)
import subprocess
import sys

if "google.colab" in sys.modules:
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "-q",
            "pynwb",
            "hdmf",
            "dandi",
            "remfile",
            "h5py",
            "fsspec",
            "aiohttp",
            "requests",
        ]
    )

In [2]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO
from remfile import File as RemoteFile

## Load Data from DANDI

We'll stream the same dataset from Week 1.

In [3]:
# Define the dataset location on DANDI
DANDISET_ID = "000059"
DANDISET_VERSION = "0.230907.2101"
ASSET_PATH = (
    "sub-MS22/"
    "sub-MS22_ses-Peter-MS22-180629-110319-concat_desc-processed_behavior+ecephys.nwb"
)

In [4]:
# Connect to DANDI and get the streaming URL
with DandiAPIClient() as client:
    dandiset = client.get_dandiset(DANDISET_ID, DANDISET_VERSION)
    asset = dandiset.get_asset_by_path(ASSET_PATH)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

print(f"Streaming from: {s3_url[:80]}...")

Streaming from: https://dandiarchive.s3.amazonaws.com/blobs/075/b32/075b32be-1c44-4e2f-8a91-3eeb...


In [5]:
# Open the NWB file for streaming
remote_file = RemoteFile(s3_url)
h5_file = h5py.File(remote_file, "r")
io = NWBHDF5IO(file=h5_file, load_namespaces=True)
nwbfile = io.read()

print(f"Session: {nwbfile.identifier}")

Session: 7931d1a7-09c5-457f-9f0a-22d952d8b818


## Load Position Data

In [6]:
# Get the behavior module and extract position
behavior_module = nwbfile.processing["behavior"]

position_interface = next(
    interface
    for name, interface in behavior_module.data_interfaces.items()
    if "position" in name.lower()
)

spatial_series = next(iter(position_interface.spatial_series.values()))

# Load position data and timestamps
position_data = spatial_series.data[:]
position_timestamps = spatial_series.timestamps[:]

x_position = position_data[:, 0]
y_position = position_data[:, 1]

print(f"Position data shape: {position_data.shape}")
print(f"Time range: {position_timestamps[0]:.1f} - {position_timestamps[-1]:.1f} s")

Position data shape: (198664, 3)
Time range: 6348.1 - 8003.6 s


## Select a Good Unit

We'll select a single well-isolated unit to analyze.

In [7]:
# Get units and filter for good quality
units_df = nwbfile.units.to_dataframe()

good_unit_mask = units_df["quality"].isin(["good", "good2"])
good_unit_indices = np.where(good_unit_mask)[0]

print(f"Total units: {len(units_df)}")
print(f"Good units: {len(good_unit_indices)}")

Total units: 1509
Good units: 172


In [8]:
# Select a unit to analyze
unit_idx = good_unit_indices[0]  # First good unit

# Get spike times for this unit
spike_times = nwbfile.units["spike_times"][unit_idx]

print(f"Unit {unit_idx}:")
print(f"  Number of spikes: {len(spike_times)}")
print(f"  Firing rate: {len(spike_times) / (spike_times[-1] - spike_times[0]):.2f} Hz")

Unit 572:
  Number of spikes: 1368
  Firing rate: 0.14 Hz


## Your Analysis Here

You now have:
- `x_position`, `y_position`: Animal's position over time
- `position_timestamps`: Time stamps for position samples
- `spike_times`: Times when the selected neuron fired
- `unit_idx`: Index of the selected unit

Try visualizing the place field or computing spatial firing rate maps!

## Cleanup

In [9]:
io.close()